In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn import preprocessing, metrics
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
import gc
from tqdm import tqdm
from scipy.sparse import csr_matrix
from  datetime import datetime, timedelta

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5mylib/trainy_0.pickle
/kaggle/input/m5mylib/add_Clean_Analysis_CA_2.csv
/kaggle/input/m5mylib/CA_1_interpolate.pickle
/kaggle/input/m5mylib/add_Clean_Analysis_CA_3.csv
/kaggle/input/m5mylib/WI_2_interpolate.pickle
/kaggle/input/m5mylib/add_Clean_Analysis_WI_1.csv
/kaggle/input/m5mylib/add_Clean_Analysis_TX_1.csv
/kaggle/input/m5mylib/CA_4_interpolate.pickle
/kaggle/input/m5mylib/TX_1_interpolate.pickle
/kaggle/input/m5mylib/WI_1_interpolate.pickle
/kaggle/input/m5mylib/CA_2_interpolate.pickle
/kaggle/input/m5mylib/add_Clean_Analysis_TX_3.csv
/kaggle/input/m5mylib/test.pickle
/kaggle/input/m5mylib/trainX_0.pickle
/kaggle/input/m5mylib/TX_2_interpolate.pickle
/kaggle/input/m5mylib/add_Clean_Analysis_WI_2.csv
/kaggle/input/m5mylib/add_Clean_Analysis_CA_1.csv
/kaggle/input/m5mylib/WI_3_interpolate.pickle
/kaggle/input/m5mylib/TX_3_interpolate.pickle
/kaggle/input/m5mylib/answear.pickle
/kaggle/input/m5mylib/merge_data2.pickle
/kaggle/input/m5mylib/add_Clean_Analysis_TX_2.cs

In [20]:
def encode_categorical(df, cols):
    
    for col in cols:
        # Leave NaN as it is.
        le = LabelEncoder()
        #not_null = df[col][df[col].notnull()]
        df[col] = df[col].fillna('nan')
        df[col] = pd.Series(le.fit_transform(df[col]), index=df.index)

    return df


#calendar = encode_categorical(
 #   calendar, ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]
#).pipe(reduce_mem_usage)

In [21]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns毎に処理
        col_type = df[col].dtypes
        if col_type in numerics: #numericsのデータ型の範囲内のときに処理を実行. データの最大最小値を元にデータ型を効率的なものに変更
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [22]:
product = pd.read_pickle("/kaggle/input/m5mylib/product.pickle").pipe(reduce_mem_usage)

Mem. usage decreased to  0.64 Mb (0.0% reduction)


In [23]:
datas = pd.read_pickle("/kaggle/input/m5mylib/merge_data2.pickle")

In [24]:
datas = datas[datas["variable"] >= 14]

In [25]:
dictionary = {0:0,1:0,2:0,3:0,4:1,5:1,6:1,7:2,8:2,9:2}
datas["state_id"] = datas["store_id"].map(dictionary)

In [7]:
datas.shape

(24392000, 36)

In [19]:
datas.columns

Index(['item_id', 'week', 'wday', 'variable', 'value', 'prev_1year', 'year',
       'date', 'store_id', 'sell_num', 'id', 'wm_yr_wk', 'event_name_1',
       'event_name_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price',
       'prev_28', 'prev_29', 'prev_30', 'p_prev_1', 'rolling_mean_t7',
       'rolling_mean_t14', 'rolling_mean_t28', 'rolling_std_t7',
       'rolling_std_t14', 'rolling_std_t28', 'price_change_t1',
       'rolling_price_std_t7', 'rolling_price_std_t28', 'quarter', 'month',
       'day', 'dayofweek', 'is_weekend'],
      dtype='object')

In [26]:
#nanが混じったデータを削除(ラグが40を超えるようなパターンだと予測期間のものまで消えてしまうので一部のみ)
#ラグが42
check_feature=[
               'sell_price', 'prev_28', 'prev_29','prev_30','p_prev_1','rolling_mean_t7', 'rolling_mean_t7','rolling_mean_t14','price_change_t1','rolling_std_t7',
               'rolling_std_t14', 'rolling_price_std_t7', 'rolling_price_std_t28', 'price_change_t1', 'rolling_price_std_t28',
               ]

In [27]:
#dataの時系列特徴列(prev_〇など)をみてnanを含むデータ削除する
#len(np.unique(datas.dropna(subset=check_feature)["id"]))
datas.dropna(subset=check_feature, inplace=True)
#len(np.unique(datas.dropna(subset=check_feature)["id"]))

In [12]:
datas.shape

(23080601, 37)

In [31]:
#ユニーク数の確認(30490あればOK)

len(np.unique(datas["id"]))

30490

In [28]:
keys = np.arange(30490)
values = np.array(product["id"])
dictionary = {k: v for k, v in zip(keys, values)}

In [29]:
# idが名前じゃないとwrmsseの重みが計算できないので名前に変換
datas["id"] = datas["id"].map(dictionary)

In [20]:
#予測期間データの確認(30490*28=853720あればOK)
datas[(datas['date'] >= '2016-03-28')& (datas['date'] <= '2016-04-24')]

,item_id,week,wday,variable,value,prev_1year,year,date,store_id,sell_num,...,rolling_std_t28,price_change_t1,rolling_price_std_t7,rolling_price_std_t28,quarter,month,day,dayofweek,is_weekend,state_id
3314263,1437,13,3,16,1.0,4.0,2016,2016-03-28,0,1.0,...,1.276367,0.0,0.0,5.960464e-08,1,3,28,0,0,0
3314264,1438,13,3,16,1.0,1.0,2016,2016-03-28,0,1.0,...,0.356445,0.0,0.0,0.000000e+00,1,3,28,0,0,0
3314265,1439,13,3,16,0.0,0.0,2016,2016-03-28,0,0.0,...,0.475586,0.0,0.0,0.000000e+00,1,3,28,0,0,0
3314266,1440,13,3,16,0.0,1.0,2016,2016-03-28,0,0.0,...,1.873047,0.0,0.0,0.000000e+00,1,3,28,0,0,0
3314267,1441,13,3,16,1.0,1.0,2016,2016-03-28,0,1.0,...,1.334961,0.0,0.0,0.000000e+00,1,3,28,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34764693,1432,16,2,16,1.0,0.0,2016,2016-04-24,9,1.0,...,0.956543,0.0,0.0,0.000000e+00,2,4,24,6,1,2
34764694,1433,16,2,16,0.0,0.0,2016,2016-04-24,9,0.0,...,0.000000,0.0,0.0,1.873779e-01,2,4,24,6,1,2
34764695,1434,16,2,16,0.0,1.0,2016,2016-04-24,9,0.0,...,0.927734,0.0,0.0,5.960464e-08,2,4,24,6,1,2
34764696,1435,16,2,16,3.0,0.0,2016,2016-04-24,9,3.0,...,1.076172,0.0,0.0,0.000000e+00,2,4,24,6,1,2


In [13]:
#WRMSSEの計算用(kernelより)

NUM_ITEMS = 30490
DAYS_PRED = 28

weight_mat = np.c_[np.ones([NUM_ITEMS,1]).astype(np.int8), # level 1
                   pd.get_dummies(product.state_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.store_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.cat_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.dept_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.state_id.astype(str) + product.cat_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.state_id.astype(str) + product.dept_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.store_id.astype(str) + product.cat_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.store_id.astype(str) + product.dept_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.item_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.state_id.astype(str) + product.item_id.astype(str),drop_first=False).astype('int8').values,
                   np.identity(NUM_ITEMS).astype(np.int8) #item :level 12
                   ].T

weight_mat_csr = csr_matrix(weight_mat)
del weight_mat; gc.collect()


def wrmsse(preds, data):
    
    # this function is calculate for last 28 days to consider the non-zero demand period
    
    # actual obserbed values / 正解ラベル
    y_true = data.get_label()
    
    y_true = y_true[-(NUM_ITEMS * DAYS_PRED):]
    preds = preds[-(NUM_ITEMS * DAYS_PRED):]
    # number of columns
    num_col = DAYS_PRED
    
    # reshape data to original array((NUM_ITEMS*num_col,1)->(NUM_ITEMS, num_col) ) / 推論の結果が 1 次元の配列になっているので直す
    #print(preds)
    #print(y_true)
    reshaped_preds = preds.reshape(num_col, NUM_ITEMS).T
    reshaped_true = np.array(y_true).reshape(num_col, NUM_ITEMS).T
    
          
    train = weight_mat_csr*np.c_[reshaped_preds, reshaped_true]
    #print(train)
    
    score = np.sum(
                np.sqrt(
                    np.mean(
                        np.square(
                            train[:,:num_col] - train[:,num_col:])
                        ,axis=1) / weight1) * weight2)
    
    return 'wrmsse', score, False

In [14]:
def weight_calc(data,product):
    
    # calculate the denominator of RMSSE, and calculate the weight base on sales amount

    sales_train_val = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')

    d_name = ['d_' + str(i+1) for i in range(1913)]

    sales_train_val = weight_mat_csr * sales_train_val[d_name].values

    # calculate the start position(first non-zero demand observed date) for each item / 商品の最初の売上日
    # 1-1914のdayの数列のうち, 売上が存在しない日を一旦0にし、0を9999に置換。そのうえでminimum numberを計算
    df_tmp = ((sales_train_val>0) * np.tile(np.arange(1,1914),(weight_mat_csr.shape[0],1)))

    start_no = np.min(np.where(df_tmp==0,9999,df_tmp),axis=1)-1

    flag = np.dot(np.diag(1/(start_no+1)) , np.tile(np.arange(1,1914),(weight_mat_csr.shape[0],1)))<1

    sales_train_val = np.where(flag,np.nan,sales_train_val)

    # denominator of RMSSE / RMSSEの分母
    weight1 = np.nansum(np.diff(sales_train_val,axis=1)**2,axis=1)/(1913-start_no)

    # calculate the sales amount for each item/level
    df_tmp = data[(data['date'] > '2016-03-27') & (data['date'] <= '2016-04-24')]
    df_tmp['sell_num'] = df_tmp['sell_num'] * df_tmp['sell_price']
    df_tmp =df_tmp.groupby(['id'])['sell_num'].apply(np.sum)
    df_tmp = df_tmp[product.id].values
    
    weight2 = weight_mat_csr * df_tmp 

    weight2 = weight2/np.sum(weight2)

    del sales_train_val
    gc.collect()
    
    return weight1, weight2

weight1, weight2 = weight_calc(datas,product)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [30]:
#x_train = datas[datas['date'] < '2016-04-18']
x_train = datas[datas['date'] < '2016-03-28']
y_train = x_train['sell_num']
#x_val = datas[datas['date'] >= '2016-04-18']
x_val = datas[(datas['date'] >= '2016-03-28')& (datas['date'] <= '2016-04-24')]
#x_val = pd.merge(product, x_val)
y_val = x_val['sell_num']
test = datas[datas['date'] > '2016-04-24']
del datas
gc.collect()

28

In [16]:
x_train.columns

Index(['item_id', 'week', 'wday', 'variable', 'value', 'prev_1year', 'year',
       'date', 'store_id', 'sell_num', 'id', 'wm_yr_wk', 'event_name_1',
       'event_name_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price',
       'prev_28', 'prev_29', 'prev_30', 'p_prev_1', 'rolling_mean_t7',
       'rolling_mean_t14', 'rolling_mean_t28', 'rolling_std_t7',
       'rolling_std_t14', 'rolling_std_t28', 'price_change_t1',
       'rolling_price_std_t7', 'rolling_price_std_t28', 'quarter', 'month',
       'day', 'dayofweek', 'is_weekend'],
      dtype='object')

In [ ]:
x_train

In [31]:
features = [
   #訓練に用いる特徴量をここに記述
    "item_id",
    #"dept_id",
    #"cat_id",
    "store_id",
    "state_id",
    "event_name_1",
    #"event_type_1",
    "event_name_2",
    #"event_type_2",
    
    
    "snap_CA",
    "snap_TX",
    "snap_WI",
    
    "sell_price",
    # demand features.
    #'prev_1',
    #'prev_2',
    #'prev_7', 
    #'prev_14', 
    'prev_28',
    'prev_29',
    'prev_30',
    #'prev_182', 
    #'prev_365', 
    #'prev_1year',
    'p_prev_1', 
    'rolling_mean_t7',
    'rolling_mean_t14', 
    'rolling_mean_t28',
    'rolling_std_t7',
    'rolling_std_t14',
    'rolling_std_t28',
    "price_change_t1",
    "rolling_price_std_t7",
    "rolling_price_std_t28",
    # time features.
    "year",
    "month",
    "week",
    "day",
    "dayofweek",
   
]

train_set = lgb.Dataset(x_train[features], y_train)
val_set = lgb.Dataset(x_val[features], y_val)

del x_train, y_train

In [32]:
#kernelから取ってきたパラメータ(objective=poisson)
params = {
        "objective" : "poisson",
        "metric" : "custom",
        #"metric" : "rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        'verbosity': 1,
        'num_iterations' : 1200,
        'num_leaves': 128,
        "min_data_in_leaf": 100,
        "random_state": 32,
}

In [33]:
#訓練
model = lgb.train(params, train_set, early_stopping_rounds = 150,
        valid_sets = [train_set, val_set], categorical_feature = ["item_id","state_id","store_id","event_name_1","event_name_2"], verbose_eval = 20, feval= wrmsse)
#valid_sets = [train_set, val_set], categorical_feature = ["item_id","dept_id","cat_id","store_id","state_id","event_name_1","event_name_2"], verbose_eval = 50, feval= wrmsse)
#カテゴリ変数は、エンコードしたかに関わらず明示しておいた方が良さそう

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['event_name_1', 'event_name_2', 'item_id', 'state_id', 'store_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 150 rounds
[20]	training's wrmsse: 1.62065	valid_1's wrmsse: 1.95245
[40]	training's wrmsse: 1.13415	valid_1's wrmsse: 1.42236


KeyboardInterrupt: 

In [18]:
#訓練
model = lgb.train(params, train_set, early_stopping_rounds = 150,
        valid_sets = [train_set, val_set], categorical_feature = ["item_id","state_id","store_id","event_name_1","event_name_2"], verbose_eval = 20, feval= wrmsse)
#valid_sets = [train_set, val_set], categorical_feature = ["item_id","dept_id","cat_id","store_id","state_id","event_name_1","event_name_2"], verbose_eval = 50, feval= wrmsse)
#カテゴリ変数は、エンコードしたかに関わらず明示しておいた方が良さそう

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['event_name_1', 'event_name_2', 'item_id', 'state_id', 'store_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 150 rounds
[20]	training's wrmsse: 1.61159	valid_1's wrmsse: 1.89178
[40]	training's wrmsse: 1.06484	valid_1's wrmsse: 1.37065
[60]	training's wrmsse: 0.823166	valid_1's wrmsse: 1.13188


KeyboardInterrupt: 